### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import os
import json
import requests
from tqdm import tqdm
import time
import keras

C:\Users\riskf\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\riskf\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\riskf\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def build_encoder(latent_dim=20):
    encoder_inputs = layers.Input(shape=(100,)) #change here features*2
    x = layers.Dense(256, activation='relu')(encoder_inputs)
    x = layers.Dense(64, activation='relu')(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
    return encoder

def build_decoder(latent_dim=20):
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(64, activation='relu')(latent_inputs)
    x = layers.Dense(256, activation='relu')(x)
    decoder_outputs = layers.Dense(100, activation='linear')(x) #change here features*2
    decoder = models.Model(latent_inputs, decoder_outputs, name="decoder")
    return decoder

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        # Initialize trackers for monitoring losses
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        # Return list of metrics to be updated during training
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def call(self, inputs, training=False):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Compute KL divergence loss even during inference to track loss correctly
        kl_loss = -0.5 * tf.reduce_mean(
            1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
        # Only add KL loss during training
        if training:
            self.add_loss(kl_loss)
        return reconstructed

    def train_step(self, data):
    # Unpack the data
        x = data[0] if isinstance(data, tuple) else data

        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(x, training=True)
            reconstruction = self.decoder(z, training=True)

            # If data is flat (e.g., shape=(batch_size, features)), adjust axis accordingly
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(x, reconstruction), axis=-1
            )
            reconstruction_loss = tf.reduce_sum(reconstruction_loss)  # Sum over all dimensions

            kl_loss = -0.5 * tf.reduce_mean(
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1
            )
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        return {'loss': total_loss, 'reconstruction_loss': reconstruction_loss, 'kl_loss': kl_loss}

# Example usage:
latent_dim = 16  
encoder = build_encoder(latent_dim)
decoder = build_decoder(latent_dim)
vae = VAE(encoder, decoder)
#vae.compile(optimizer='adam')
vae.compile(optimizer=tf.keras.optimizers.Adam())


In [3]:
#relative paths. # Set directory paths for later use.
# Get the directory of the script file
base_dir = os.getcwd()
base_dir
ligants_type=['enzyme','GPCR','ion_channel','nuclear_receptor']
ltype=ligants_type[0]
file_name='final_new_par_LMF_50.csv'
file_path = os.path.join(base_dir,'data','split',ltype, file_name)
output_path = file_path
data_frame = pd.read_csv(file_path, header=None, skiprows=1)


In [4]:
# Filter to only include instances with label 1 (interactions)
# Separate features and labels
filtered_df = data_frame[data_frame.iloc[:, -1] == 1]  # All rows, all columns except the last one
features_new = filtered_df.iloc[:, :-1]     # All rows, just the last column

# Convert features DataFrame to a NumPy array if necessary
x_train = features_new.to_numpy()

In [5]:
#para
epochs=4
batch_size=77

In [6]:
# Train the model
vae.fit(x_train, epochs=epochs, batch_size=batch_size)

Epoch 1/4
38/38 [==============================] - 1s 2ms/step - loss: -2.2365 - reconstruction_loss: -2.7872 - kl_loss: 0.1030
Epoch 2/4
38/38 [==============================] - 0s 2ms/step - loss: -2.9018 - reconstruction_loss: -2.6394 - kl_loss: 0.0451
Epoch 3/4
38/38 [==============================] - 0s 2ms/step - loss: -2.9901 - reconstruction_loss: -3.0472 - kl_loss: 0.0569
Epoch 4/4
38/38 [==============================] - 0s 2ms/step - loss: -2.9756 - reconstruction_loss: -2.8316 - kl_loss: 0.0294


In [7]:
# Filter to only include instances with label 1 (interactions)
filtered_df = data_frame[data_frame.iloc[:, -1] == 1]

# Separate features
features_new = filtered_df.iloc[:, :-1]  # Assuming the last column is the label

In [8]:
x_new = features_new.to_numpy()

In [9]:
z_mean, z_log_var, z = vae.encoder.predict(x_new)
# Now, z contains the latent representations of  filtered data.

92/92 [==============================] - 0s 681us/step


In [10]:
reconstructed_new = vae.predict(x_new)

92/92 [==============================] - 0s 793us/step


In [11]:
# Convert the reconstructed data to a DataFrame
reconstructed_df = pd.DataFrame(reconstructed_new)

# Display the first few rows of the reconstructed DataFrame
print(pd.DataFrame(x_new))
print(reconstructed_df)

            0         1         2         3         4         5         6   \
0     0.378580  0.279222  0.529452  0.224391  0.269244  0.288321  0.116587   
1     0.378580  0.279222  0.529452  0.224391  0.269244  0.288321  0.116587   
2     0.191642  0.174801  0.150674  0.324245  0.216482  0.277361  0.236950   
3     0.191642  0.174801  0.150674  0.324245  0.216482  0.277361  0.236950   
4     0.234871  0.244009  0.164632  0.376041  0.268878  0.334274  0.310694   
...        ...       ...       ...       ...       ...       ...       ...   
2921  0.457356  0.257199  0.352959  0.327653  0.473576  0.241599  0.613059   
2922  0.457356  0.257199  0.352959  0.327653  0.473576  0.241599  0.613059   
2923  0.256248  0.247317  0.186627  0.386409  0.290559  0.357915  0.350838   
2924  0.250683  0.227646  0.241334  0.399093  0.316098  0.338614  0.326268   
2925  0.233042  0.225892  0.219956  0.389313  0.257460  0.331327  0.301114   

            7         8         9   ...        90        91    

In [12]:
# Add a new column 'Label' with all values set to 1
reconstructed_df[100] = 1

# Display the first few rows to verify the new column
print(reconstructed_df.head())

        0         1         2         3         4         5         6    \
0  0.233711  0.824533 -0.361689  0.213427  0.176328  0.361833  0.237922   
1  0.157646  0.453922 -0.714357  0.419542  0.153146  0.225082  0.183694   
2  0.203672  0.346812 -0.238632  0.300500  0.167312  0.252384  0.288623   
3  0.413539  0.252494 -0.331688  0.291488  0.244147  0.382248  0.385660   
4 -0.183347  0.544884 -0.565753  0.478404  0.154922  0.333349  0.255030   

        7         8         9    ...       91        92        93        94   \
0  0.275631  0.469756  0.133133  ... -0.616203 -0.646469 -0.442575 -0.580871   
1  0.335473  0.343401  0.396220  ... -0.590101 -0.692909 -0.577921 -0.501927   
2  0.340246  0.256751  0.368965  ... -0.391984 -0.357546 -0.317578 -0.278554   
3  0.657324  0.326355  0.214861  ... -0.302604 -0.796272 -0.494969 -0.351633   
4  0.552091  0.475044  0.406952  ... -0.880203 -0.819833 -0.505979 -0.240180   

        95        96        97        98        99   100  
0 -0.3250

In [13]:
num_samples_to_generate = 289628  # The number of new rows want to generate

# Generate random samples from the latent space
latent_dim = 16  # Ensure this matches the latent dimension size of  VAE
z_new_samples = np.random.normal(size=(num_samples_to_generate, latent_dim))

# Use the decoder to generate new data
new_data_generated = vae.decoder.predict(z_new_samples)



9051/9051 [==============================] - 5s 587us/step


In [14]:
new_data_generated

array([[ 0.09607969,  0.47993645, -0.23520415, ..., -0.37171686,
        -0.48678243, -0.42768505],
       [ 0.18671435,  0.42267683, -0.5109693 , ..., -0.2641505 ,
        -0.36249384, -0.18440033],
       [ 0.47839245,  0.2465533 , -0.58737195, ..., -0.4443998 ,
        -0.49874362, -0.48620078],
       ...,
       [ 0.16239291,  0.42153484, -0.80414003, ..., -0.20427476,
        -0.45685947, -0.5756828 ],
       [ 0.16119185,  0.5059398 , -0.43641225, ..., -0.2997183 ,
        -0.15366216, -0.39828062],
       [ 0.6129894 ,  0.28827775, -0.5283671 , ..., -0.48132578,
        -0.30079854, -0.4160057 ]], dtype=float32)

In [15]:
# Convert the generated data to a DataFrame
new_data_df = pd.DataFrame(new_data_generated)

# Add a column 'Label' with all values set to 1
new_data_df[100] = 1

In [16]:
enhanced_df = pd.concat([data_frame, new_data_df], axis=0).reset_index(drop=True)

In [17]:
enhanced_df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.378580,0.279222,0.529452,0.224391,0.269244,0.288321,0.116587,0.395598,0.339238,0.202399,...,-0.640229,0.179676,-0.025780,0.637873,-0.032854,-0.968462,0.351491,-0.433654,0.828018,1.0
1,0.378580,0.279222,0.529452,0.224391,0.269244,0.288321,0.116587,0.395598,0.339238,0.202399,...,-0.038291,0.038372,-0.141714,-0.202433,-0.500281,-0.184908,-0.386320,-0.435208,-0.225072,0.0
2,0.378580,0.279222,0.529452,0.224391,0.269244,0.288321,0.116587,0.395598,0.339238,0.202399,...,-0.356340,-0.321038,-0.323238,-0.444865,-0.348860,-0.419126,-0.456461,-0.397482,-0.391028,0.0
3,0.378580,0.279222,0.529452,0.224391,0.269244,0.288321,0.116587,0.395598,0.339238,0.202399,...,-0.339246,-0.312920,-0.305780,-0.427002,-0.366420,-0.416211,-0.452622,-0.389398,-0.394058,0.0
4,0.378580,0.279222,0.529452,0.224391,0.269244,0.288321,0.116587,0.395598,0.339238,0.202399,...,-0.332402,-0.331912,-0.333846,-0.432593,-0.318621,-0.380771,-0.445847,-0.366790,-0.433988,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585103,0.147669,0.541537,-0.181800,0.492964,0.120922,0.391678,0.144622,0.197834,0.325198,0.299830,...,-0.355880,-0.404848,-0.239798,-0.269187,-0.457925,-0.365233,-0.249292,-0.273733,-0.454504,1.0
585104,0.121502,0.260829,-0.248446,0.422976,0.205730,0.233995,0.310397,0.360269,0.322920,0.463311,...,-0.434099,-0.495137,-0.549230,-0.453075,-0.327035,-0.357408,-0.080530,-0.381289,-0.427422,1.0
585105,0.162393,0.421535,-0.804140,0.450230,0.187825,0.219477,0.307229,0.300398,0.403289,0.081054,...,-0.365379,-0.496839,-0.281589,-0.454220,-0.478565,-0.613799,-0.204275,-0.456859,-0.575683,1.0
585106,0.161192,0.505940,-0.436412,0.375269,0.228820,0.334090,0.204048,0.381848,0.404161,0.291552,...,-0.696127,-0.388773,-0.351264,-0.331284,-0.493648,-0.445110,-0.299718,-0.153662,-0.398281,1.0


In [18]:
file_name='enhanced_VAE_final_new_par_50_LFM_space_3.csv'
file_path = os.path.join(base_dir,'data','split',ltype, file_name)
output_path = file_path
enhanced_df.to_csv(output_path, index=False)